## Init e dependencies

In [11]:
! pip install langchain==0.2.7 langchain_aws langchain-community langchain_core boto3 botocore
! pip install PyMuPDF python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
! pip install pymupdf python-docx pydantic

Defaulting to user installation because normal site-packages is not writeable


### llm endpoints (to edit)

In [9]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import urllib3
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from google.colab import userdata
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name='eu-west-1',
        aws_access_key_id=userdata.get('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=userdata.get('AWS_SECRET_ACCESS_KEY'),
        verify=False,  # Disable SSL verification
        config=Config(
            proxies={'https': None}
        )
    )

llm = ChatBedrock(
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        client=bedrock_client,
        model_kwargs={
            "temperature": 0,
            "max_tokens": 2000,
        }
    )


In [16]:
import fitz  # PyMuPDF
import docx

from google.colab import files

def upload_file():
  uploaded = files.upload()
  # Una volta eseguita questa cella, Colab ti chiederà di selezionare il file dal tuo PC.

  # 2) Identifica il nome del file caricato
  for file_name in uploaded.keys():
      file_path = file_name

  return file_path

def read_pdf(file_path):
    doc = fitz.open(file_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = "\n".join([p.text for p in doc.paragraphs])
    return text

def read_txt(file_path):
    with open(file_path, "r") as f:
        text = f.read()
    return text

def upload_and_read_pdf():
    file_path = upload_file()

    return read_pdf(file_path)

def upload_and_read_docx():
    file_path = upload_file()

    return read_docx(file_path)

def upload_and_read_txt():
    file_path = upload_file()

    return read_txt(file_path)

In [17]:
def generate(text):

    prompt_template = PromptTemplate(
        input_variables=["protagonist", "setting"],
        template="""
            Riassumi il seguente testo:
            {text}

            REGOLE
            - scrivi una breve intro per spiegare il contesto
            - riassumi i punti principali in una bullet list
            - trai le tue conclusioni sull'argomento
        """
    )

    chain = prompt_template | llm | StrOutputParser()

    response = chain.invoke({"text": text})
    return response


In [18]:

text = upload_and_read_pdf()

response = generate(text)

Saving RomeCall_Paper_web.pdf to RomeCall_Paper_web (1).pdf


In [19]:
from IPython.display import display, Markdown
display(Markdown(f"### Riassunto:\n\n{response}"))


### Riassunto:

Introduzione
Il testo presenta la "Rome Call for AI Ethics", un documento firmato nel 2020 per promuovere un approccio etico all'intelligenza artificiale. L'obiettivo è di creare un futuro in cui l'innovazione digitale e il progresso tecnologico mettano l'essere umano al centro.

Punti principali:
- L'IA offre grandi potenzialità per migliorare il benessere sociale e personale, ma deve essere sviluppata nel rispetto della dignità umana e senza avere come unico scopo il profitto.
- I firmatari si impegnano a promuovere uno sviluppo dell'IA al servizio dell'intera umanità, che non discrimini nessuno e che sia attento all'ambiente.
- Vengono definiti 6 principi etici fondamentali per l'IA: trasparenza, inclusione, responsabilità, imparzialità, affidabilità, sicurezza e privacy.
- Viene sottolineata l'importanza dell'istruzione e della formazione per sensibilizzare sulle opportunità e sui rischi dell'IA.
- Sono necessarie nuove forme di regolamentazione per promuovere la trasparenza e il rispetto dei principi etici, soprattutto per le tecnologie ad alto rischio di impatto sui diritti umani.

Conclusioni
Il documento rappresenta un importante passo avanti nella definizione di un quadro etico per lo sviluppo dell'intelligenza artificiale, con l'obiettivo di garantire che essa serva l'intera umanità e il pianeta, nel rispetto della dignità e dei diritti di ogni essere umano. L'impegno congiunto di istituzioni, aziende e società civile è fondamentale per realizzare questa visione.

In [20]:
text = upload_and_read_docx()

summary = generate(text)

display(Markdown(f"### Riassunto:\n\n{response}"))

Saving football_league.docx to football_league.docx


### Riassunto:

Introduzione
Il testo presenta la "Rome Call for AI Ethics", un documento firmato nel 2020 per promuovere un approccio etico all'intelligenza artificiale. L'obiettivo è di creare un futuro in cui l'innovazione digitale e il progresso tecnologico mettano l'essere umano al centro.

Punti principali:
- L'IA offre grandi potenzialità per migliorare il benessere sociale e personale, ma deve essere sviluppata nel rispetto della dignità umana e senza avere come unico scopo il profitto.
- I firmatari si impegnano a promuovere uno sviluppo dell'IA al servizio dell'intera umanità, che non discrimini nessuno e che sia attento all'ambiente.
- Vengono definiti 6 principi etici fondamentali per l'IA: trasparenza, inclusione, responsabilità, imparzialità, affidabilità, sicurezza e privacy.
- Viene sottolineata l'importanza dell'istruzione e della formazione per sensibilizzare sulle opportunità e sui rischi dell'IA.
- Sono necessarie nuove forme di regolamentazione per promuovere la trasparenza e il rispetto dei principi etici, soprattutto per le tecnologie ad alto rischio di impatto sui diritti umani.

Conclusioni
Il documento rappresenta un importante passo avanti nella definizione di un quadro etico per lo sviluppo dell'intelligenza artificiale, con l'obiettivo di garantire che essa serva l'intera umanità e il pianeta, nel rispetto della dignità e dei diritti di ogni essere umano. L'impegno congiunto di istituzioni, aziende e società civile è fondamentale per realizzare questa visione.